In [ ]:


import sys
import nbformat
import pandas as pd
import plotly.graph_objects as go

# Add parent directory to sys.path so 'src' can be imported
sys.path.append("../")

from src.data_loader import download_stock_data
from src.support_resistance import find_support_resistance

# Load fresh clean DataFrame
df = download_stock_data("RELIANCE.NS", period="6mo")
df.reset_index(drop=True, inplace=True)

# Convert 'Date' to datetime if needed
df['Date'] = pd.to_datetime(df['Date'])

# Flatten MultiIndex columns if present
if isinstance(df.columns, pd.MultiIndex):
    df_flat = df.copy()
    df_flat.columns = ['_'.join([str(i) for i in col if i]) for col in df.columns.values]
else:
    df_flat = df

# Rename columns to match expected names for find_support_resistance
df_sr = df_flat.rename(columns={
    'Low_RELIANCE.NS': 'Low',
    'High_RELIANCE.NS': 'High',
    'Open_RELIANCE.NS': 'Open',
    'Close_RELIANCE.NS': 'Close',
    'Volume_RELIANCE.NS': 'Volume',
    'Date_': 'Date'
})

# Find support/resistance
sr_points = find_support_resistance(df_sr, order=5)

# Convert SR Dates too
sr_points['Date'] = pd.to_datetime(sr_points['Date'])

# Drop any bad data just in case
sr_points = sr_points.dropna(subset=['Date', 'Price'])

# Plot
fig = go.Figure()

# Add candlestick chart
fig.add_trace(go.Candlestick(
    x=df["Date"],
    open=df["Open"],
    high=df["High"],
    low=df["Low"],
    close=df["Close"],
    name="Candlestick",
    increasing_line_color='green',
    decreasing_line_color='red'
))

# Add SR points
for _, row in sr_points.iterrows():
    fig.add_trace(go.Scatter(
        x=[row["Date"]],
        y=[row["Price"]],
        mode="markers+text",
        name=row["Type"],
        text=[row["Type"]],
        textposition="top center",
        marker=dict(
            size=8,
            color="green" if row["Type"] == "Support" else "red",
            symbol="x"
        )
    ))


fig.update_layout(
    title="Support and Resistance — RELIANCE.NS",
    xaxis_rangeslider_visible=False,
    height=600,
    template="plotly_white",
    yaxis_title="Price (INR)",
    xaxis_title="Date"
)

fig.show()
# Save to processed directory (create if doesn't exist)
import os
os.makedirs("../data/processed", exist_ok=True)

# Save CSV
sr_points.to_csv("../data/processed/reliance_support_resistance.csv", index=False)
print("✅ Support/Resistance CSV saved at data/processed/reliance_support_resistance.csv")
import os

# Ensure the 'data/processed' folder exists
os.makedirs("../data/processed", exist_ok=True)

# Save SR points to CSV
csv_path = "../data/processed/reliance_support_resistance.csv"
sr_points.to_csv(csv_path, index=False)

print(f"✅ Support/Resistance CSV saved at {csv_path}")


d:\BullsandBears\BullsandBears\notebooks\..\src\data_loader.py:7: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed

Saved to data/raw/


✅ Support/Resistance CSV saved at data/processed/reliance_support_resistance.csv
✅ Support/Resistance CSV saved at ../data/processed/reliance_support_resistance.csv
